In [ ]:
# ==============================================================================
# 📦 0. INSTALACE A NAČTENÍ KNIHOVEN (START)
# ==============================================================================

# Seznam potřebných balíčků
required_packages <- c("readxl", "dplyr", "openxlsx", "rstatix", "tidyverse", "moments", "vcd")

# Funkce: Nainstaluj jen to, co chybí, a pak vše načti
install_and_load <- function(pkg) {
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg, dependencies = TRUE)
    library(pkg, character.only = TRUE)
  }
}

# Spuštění pro všechny balíčky najednou
lapply(required_packages, install_and_load)

# Pro jistotu explicitní načtení (abys viděl, že to funguje):
library(readxl)    # Čtení Excelů
library(openxlsx)  # Zápis Excelů
library(tidyverse) # Obsahuje dplyr, ggplot2, tidyr (na filtrace a grafy)
library(rstatix)   # Statistické testy (moderní verze)
library(moments)   # Šikmost a špičatost (skewness, kurtosis)
library(vcd)       # Kategoriální data (Cramerovo V, Mozaikové grafy)

print("✅ Všechny knihovny jsou úspěšně načteny!")

In [ ]:
# Načtení dat z excelu -> Název souboru ||     Sheet         || skip řádků
data <- readxl::read_excel("data_hraci.xlsx", sheet = "hraci", skip = 0)
head(data)

In [ ]:
# POKUD DATA POTŘEBUJI PREPROCESOVAT, NEMÁM JE TREBA ROZDĚLENÉ...

# ==============================================================================
# 🔄 UNIVERZÁLNÍ PIVOT (WIDE -> LONG) - TAHÁK
# ==============================================================================
# Kdy použít: Když máš měření ve sloupcích (rok_2018, rok_2019) a chceš je pod sebe.
# Co upravit:
#   1. cols -> Které sloupce se mají "smrsknout" (ty co obsahují měřená data)
#   2. names_to -> Název nového sloupce pro kategorii (např. "Rok", "Mesic")
#   3. values_to -> Název nového sloupce pro hodnoty (např. "Pocet", "Vykon")
#   4. names_prefix -> (Volitelné) Text, který chceš umazat z názvu sloupců
# ==============================================================================

# Předpoklad: data jsou načtená v proměnné 'data'

df_long <- data %>%
  pivot_longer(
    # --- 1. VÝBĚR SLOUPCŮ (Odkomentuj jen jednu možnost) ---

    # Možnost A: Vyjmenovat konkrétní sloupce (NEJBEZPEČNĚJŠÍ)
    # Použij, když je sloupců málo a víš přesně, jak se jmenují.
    cols = c(odehrane_hod_2018, odehrane_hod_2019),
  
    # Možnost B: Všechno KROMĚ identifikačních sloupců (NEJRYCHLEJŠÍ)
    # Použij, když máš mraky sloupců s daty a jen pár IDček (ID, jméno, system).
    # cols = -c(IDhrace, system),
    
    # Možnost C: Sloupce začínající určitým textem (CHYTRÉ)
    # cols = starts_with("odehrane_hod_"),

    # --------------------------------------------------------
    
    # --- 2. NASTAVENÍ NÁZVŮ ---
    names_to = "Rok",       # Jak nazvat sloupec, kam spadnou "2018", "2019"
    values_to = "Hodiny",   # Jak nazvat sloupec, kam spadnou čísla
    
    # --- 3. ČIŠTĚNÍ NÁZVŮ (Odkomentuj podle potřeby) ---
    
    # Situace A: Jednoduchý prefix (Tvůj aktuální případ)
    # Máš "odehrane_hod_2018" -> chceš jen "2018".
    names_prefix = "odehrane_hod_"
    
    # Situace B: Složitější vzor (Regulární výrazy) - PROFI
    # Pokud bys měl např. "Karta_Nvidia_FPS" a chtěl vytáhnout Kartu i Metriku.
    # names_pattern = "(.*)_average_(.*)", 
    # names_to = c("Karta", "Metrika") 
  ) 

# --- 4. DODATEČNÉ ÚPRAVY (Volitelné) ---

# Převod Roku na číslo (protože z názvu sloupce se vždy stane text)
# df_long$Rok <- as.numeric(df_long$Rok)

# Kontrola výsledku
head(df_long)

In [ ]:
# ==============================================================================
# 🛠️ MANIPULACE S DATY (Tvorba a úprava sloupců)
# ==============================================================================
# Kdy použít: "Vytvořte sloupec XY...", "Převeďte na kategorie...", "Očistěte názvy..."
# ==============================================================================

# 1. TVORBA NOVÉHO ČÍSELNÉHO SLOUPCE (Matematika)
# ------------------------------------------------
data_upravena <- data %>%
  mutate(
    # Příklad: Rozdíl dvou sloupců
    Zmena_Vahy = Vaha_Po - Vaha_Pred,
    
    # Příklad: Logaritmus (časté u mezd/cen pro normalizaci)
    Log_Vynos = log(Rocni_Vynosnost_v_Procentech),
    
    # Příklad: Převod jednotek (např. minuty na hodiny)
    Cas_Hodiny = Cas_Minuty / 60
  )


# 2. KATEGORIZACE (Tvorba slovního sloupce z čísel) - PODMÍNKY
# ------------------------------------------------
# Příklad: "Pokud je výnos > 0, je to Zisk, jinak Ztráta"
data_upravena <- data %>%
  mutate(
    # A) Jednoduchá podmínka (ifelse)
    Vysledek_Text = ifelse(Rocni_Vynosnost_v_Procentech > 0, "Zisk", "Ztrata"),
    
    # B) Složitější podmínka (case_when) - SUPER UŽITEČNÉ!
    # Příklad: Věkové skupiny nebo Hodnocení
    Vekova_Skupina = case_when(
      Vek < 18 ~ "Junior",
      Vek >= 18 & Vek < 65 ~ "Dospely",
      Vek >= 65 ~ "Senior",
      TRUE ~ "Neznámé"  # Pojistka pro ostatní/chybějící
    )
  )


# 3. PŘEJMENOVÁNÍ A PŘEUSPOŘÁDÁNÍ SLOUPCŮ
# ------------------------------------------------
data_final <- data_upravena %>%
  # Přejmenování: Novy_Nazev = Stary_Nazev
  rename(
    ID = `ID Hrace`,       # Zbavíme se mezer v názvu
    Vynos = Rocni_Vynosnost_v_Procentech
  ) %>%
  # Výběr jen toho, co chci (a seřazení sloupců)
  select(ID, Vynos, Vysledek_Text, everything()) # everything() dá zbytek nakonec


# 4. FILTROVÁNÍ A ŘAZENÍ (Jen pro pořádek)
# ------------------------------------------------
# data_sorted <- data_final %>%
#   filter(Vynos > 0) %>%        # Jen ziskové
#   arrange(desc(Vynos))         # Seřadit od největšího

In [ ]:
# ==============================================================================
# 📊 POPISNÁ STATISTIKA (METRIKY) - TAHÁK
# ==============================================================================
# Kdy použít: "Určete průměr, medián, směrodatnou odchylku, variační koeficient..."
#             "Jsou data sešikmená?"
# ==============================================================================

library(moments) # Nutné pro šikmost (skewness) a špičatost (kurtosis)

# 1. VÝBĚR PROMĚNNÉ (Tady jen změň název sloupce!)
# ------------------------------------------------
x <- data$odehrane_hod_2018  # <--- SEM HOĎ TEN SLOUPEC, CO CHCEŠ SPOČÍTAT
# x <- data_hraci$vek        # Příklad

# 2. VÝPOČET VŠECH METRIK NAJEDNOU
# ------------------------------------------------
# (na.rm = TRUE zajistí, že to nespadne, když tam chybí nějaké číslo)

# --- Míry polohy ---
prumer  <- mean(x, na.rm = TRUE)
median_x <- median(x, na.rm = TRUE)
min_x   <- min(x, na.rm = TRUE)
max_x   <- max(x, na.rm = TRUE)

# --- Míry variability ---
rozptyl <- var(x, na.rm = TRUE)      # Variance (s^2)
smer_odchylka <- sd(x, na.rm = TRUE) # Standard Deviation (s)
rozpeti <- max_x - min_x             # Range (R)
iqr_val <- IQR(x, na.rm = TRUE)      # Mezikvartilové rozpětí (Q3 - Q1)

# Variační koeficient (CV) - v procentech!
# Říká, jak moc data kolísají vzhledem k průměru. (nad 50 % = vysoká variabilita)
var_koef <- (smer_odchylka / abs(prumer)) * 100 

# --- Míry tvaru (Normální rozdělení má šikmost 0 a špičatost 3) ---
sikmost <- skewness(x, na.rm = TRUE)  # Skewness (>0 sešikmení doprava, <0 doleva)
spicatost <- kurtosis(x, na.rm = TRUE) # Kurtosis (Standardně v R: Normální = 3)

# --- Kvantily ---
kvantily <- quantile(x, probs = c(0.10, 0.25, 0.50, 0.75, 0.90), na.rm = TRUE)


# 3. PŘEHLEDNÝ VÝPIS (Tohle jen spusť a opisuj čísla)
# ------------------------------------------------
print("--- MÍRY POLOHY ---")
print(paste("Průměr (Mean):", round(prumer, 4)))
print(paste("Medián (Median):", round(median_x, 4)))
print(paste("Min:", round(min_x, 4), "| Max:", round(max_x, 4)))

print("--- MÍRY VARIABILITY ---")
print(paste("Směrodatná odchylka (SD):", round(smer_odchylka, 4)))
print(paste("Rozptyl (Var):", round(rozptyl, 4)))
print(paste("Variační koeficient (CV):", round(var_koef, 2), "%"))
print(paste("Rozpětí (Max-Min):", round(rozpeti, 4)))
print(paste("IQR (Q3-Q1):", round(iqr_val, 4)))

print("--- MÍRY TVARU ---")
print(paste("Šikmost (Skewness):", round(sikmost, 4)))
print(paste("Špičatost (Kurtosis):", round(spicatost, 4)))
# Pozn: Někdy se používá "Exces špičatosti" = Kurtosis - 3. Pokud to vyjde kolem 0, je to normální.

print("--- KVANTILY ---")
print(kvantily)

In [ ]:
# ==============================================================================
# 📊 KATEGORIÁLNÍ DATA (ČETNOSTI)
# ==============================================================================
kat_promenna <- data$system  # <--- Vyber sloupec s textem (kategorií)

# Absolutní četnosti (počty kusů)
tabulka <- table(kat_promenna)
print(tabulka)

# Relativní četnosti (procenta)
procenta <- prop.table(tabulka) * 100
print(round(procenta, 2))

# Modus (Která kategorie je nejčastější?)
modus <- names(sort(tabulka, decreasing = TRUE)[1])
print(paste("Modus (nejčastější hodnota):", modus))

In [ ]:
# ==============================================================================
# 🕵️‍♂️ DETEKCE ODLEHLÝCH HODNOT (Vnitřní hradby / IQR metoda)
# ==============================================================================
# Kdy použít: "Identifikujte odlehlá pozorování...", "Vypočtěte průměr po očištění..."
# ==============================================================================

# 1. VÝBĚR DAT (Jen změň název sloupce)
# ------------------------------------------------
# Příklad: Chci řešit sloupec 'odehrane_hod_2018'
# x <- data$odehrane_hod_2018
# NEBO s filtrem (např. jen pro Windows):
x <- data %>% 
  filter(system == "WIN") %>%  # <--- ZKONTROLUJ PŘESNÝ NÁZEV KATEGORIE!
  pull(odehrane_hod_2018)      # pull() vytáhne sloupec jako vektor čísel

# 2. VÝPOČET HRADEN (MEZÍ)
# ------------------------------------------------
# na.rm = TRUE je pojistka proti prázdným buňkám
Q1 <- quantile(x, 0.25, na.rm = TRUE)
Q3 <- quantile(x, 0.75, na.rm = TRUE)
IQR_hodnota <- IQR(x, na.rm = TRUE)    # Rozdíl Q3 - Q1

# Výpočet vnitřních hradeb (1.5 násobek IQR)
DM <- Q1 - 1.5 * IQR_hodnota # Dolní mez (Lower Fence)
HM <- Q3 + 1.5 * IQR_hodnota # Horní mez (Upper Fence)

print("--- HRANICE (Vnitřní hradby) ---")
print(paste("Dolní mez (DM):", round(DM, 4)))
print(paste("Horní mez (HM):", round(HM, 4)))
print(paste("Interval normálních hodnot: <", round(DM, 2), ";", round(HM, 2), ">"))

# 3. IDENTIFIKACE ODLEHLÝCH POZOROVÁNÍ
# ------------------------------------------------
# Hodnoty, které jsou MENŠÍ než DM nebo VĚTŠÍ než HM
odlehle <- x[x < DM | x > HM]

print("--- ODLEHLÉ HODNOTY (Outliers) ---")
if (length(odlehle) > 0) {
  print(odlehle)
  print(paste("Počet odlehlých:", length(odlehle)))
} else {
  print("Žádná odlehlá pozorování nebyla nalezena.")
}

# 4. VÝPOČET STATISTIK PO OČIŠTĚNÍ (BEZ ODLEHLÝCH)
# ------------------------------------------------
# Vybereme jen to, co je UVNITŘ hradeb
x_clean <- x[x >= DM & x <= HM]

print("--- STATISTIKY PO ODSTRANĚNÍ ODLEHLÝCH ---")
print(paste("Nový průměr:", round(mean(x_clean, na.rm = TRUE), 4)))
print(paste("Nový medián:", round(median(x_clean, na.rm = TRUE), 4)))
print(paste("Nový horní kvartil (Q3):", round(quantile(x_clean, 0.75, na.rm = TRUE), 4)))
print(paste("Nová směrodatná odchylka:", round(sd(x_clean, na.rm = TRUE), 4)))

In [ ]:
# ==============================================================================
# 1. JEDNOVÝBĚROVÝ TEST O PRŮMĚRU (t-test / Wilcoxon)
# ==============================================================================
# Kdy použít: "Je průměrná hodnota statisticky významně odlišná od 100?"
# Data: Jeden číselný sloupec.
# ==============================================================================

# 1. NASTAVENÍ
# ---------------------
x <- data$odehrane_hod_2018  # <--- TVOJE DATA
mu_0 <- 150                  # <--- ČÍSLO, SE KTERÝM SROVNÁVÁŠ (z testu)
hladina_alpha <- 0.05        # 5% hladina významnosti

# Jaký je směr testu? (Doplň do 'alternative')
# "two.sided" -> H1: Průměr se NEROVNÁ mu_0
# "greater"   -> H1: Průměr je VĚTŠÍ než mu_0
# "less"      -> H1: Průměr je MENŠÍ než mu_0
smer <- "two.sided" 

# 2. OVĚŘENÍ NORMALITY (Shapiro-Wilk)
# ---------------------
# H0: Data mají normální rozdělení. (Chceme p > 0.05)
shapiro <- shapiro.test(x)
print(paste("Shapiro p-value:", format.pval(shapiro$p.value, digits=4)))

# 3. VÝBĚR TESTU A PROVEDENÍ
# ---------------------
if(shapiro$p.value > 0.05) {
  print("✅ Data jsou normální -> Používám T-TEST")
  
  test_result <- t.test(x, mu = mu_0, alternative = smer, conf.level = 1 - hladina_alpha)
  
} else {
  print("⚠️ Data NEJSOU normální -> Používám WILCOXONŮV TEST")
  
  test_result <- wilcox.test(x, mu = mu_0, alternative = smer, conf.level = 1 - hladina_alpha)
}

# 4. VÝSLEDEK
# ---------------------
print(test_result)
p_val <- test_result$p.value

print(paste("P-hodnota:", format.pval(p_val, digits=5)))

if(p_val < hladina_alpha) {
  print(paste("✅ ZAMÍTÁME H0. Průměr je statisticky významně", smer, "než", mu_0))
} else {
  print(paste("❌ NEZAMÍTÁME H0. Neprokázal se rozdíl oproti", mu_0))
}


# ==============================================================================
# 2. JEDNOVÝBĚROVÝ TEST O PODÍLU (Procenta)
# ==============================================================================
# Kdy použít: "Je podíl úspěšných portfolií větší než 70 %?"
# Data: Sloupec s hodnotami (např. Zisk/Ztráta) nebo už spočítané počty.
# ==============================================================================

# 1. PŘÍPRAVA DAT
# ---------------------
# Musíme spočítat 'x' (úspěchy) a 'n' (celkem)
# Příklad: Úspěch je hodnota > 0
pocet_uspechu <- sum(data$Rocni_Vynosnost_v_Procentech > 0, na.rm=TRUE) 
pocet_celkem  <- sum(!is.na(data$Rocni_Vynosnost_v_Procentech))

# Parametry ze zadání
p_0  <- 0.70          # <--- TESTOVANÁ HRANICE (např. 70%)
smer <- "greater"     # "greater" (větší), "less" (menší), "two.sided" (liší se)

# 2. PROVEDENÍ TESTU (prop.test)
# ---------------------
# Pokud je dat málo (n < 30), je lepší binom.test(), jinak prop.test()
if(pocet_celkem < 30) {
  print("Malý vzorek -> Binomický test")
  test_prop <- binom.test(x = pocet_uspechu, n = pocet_celkem, p = p_0, alternative = smer)
} else {
  print("Velký vzorek -> Aproximace (prop.test)")
  test_prop <- prop.test(x = pocet_uspechu, n = pocet_celkem, p = p_0, alternative = smer)
}

# 3. VÝPIS VÝSLEDKŮ
# ---------------------
print(test_prop)
print(paste("Bodový odhad (naměřená %):", round(test_prop$estimate, 4)))
print(paste("P-hodnota:", format.pval(test_prop$p.value, digits=5)))

if(test_prop$p.value < 0.05) {
  print("✅ ZAMÍTÁME H0 -> Podíl se statisticky významně liší.")
} else {
  print("❌ NEZAMÍTÁME H0 -> Rozdíl není statisticky významný.")
}

In [ ]:
# ==============================================================================
# 🅰️ NEZÁVISLÉ VÝBĚRY (Two-sample Independent Test)
# ==============================================================================
# Kdy použít: "Liší se průměrný plat mužů a žen?", "Je Strategie A lepší než B?"
# ==============================================================================

# 1. PŘÍPRAVA DAT (Filtr na dvě skupiny)
# ---------------------
# Musíme mít jen dvě skupiny. Pokud jich je víc, vyfiltrujeme je.
data_2skupiny <- data %>% 
  filter(Strategie %in% c("A", "B")) %>%  # <--- ZMĚŇ NÁZVY SKUPIN
  droplevels() # Zahodí nepoužité kategorie

# Definice proměnných pro test
hodnota <- data_2skupiny$Rocni_Vynosnost_v_Procentech # <--- ČÍSELNÁ PROMĚNNÁ
skupina <- data_2skupiny$Strategie                    # <--- ROZDĚLOVACÍ PROMĚNNÁ (Kategorie)

# 2. OVĚŘENÍ PŘEDPOKLADŮ
# ---------------------
# a) Normalita (Shapiro) - testujeme pro KAŽDOU skupinu zvlášť
shapiro_group <- data_2skupiny %>%
  group_by(Strategie) %>%
  rstatix::shapiro_test(Rocni_Vynosnost_v_Procentech)

print("--- TEST NORMALITY (Shapiro) ---")
print(shapiro_group)
# Pokud je ve sloupci 'p' hodnota < 0.05 -> NENÍ normální -> Jdi na Mann-Whitney
# Pokud je všude 'p' > 0.05 -> JE normální -> Jdi na krok b)

# b) Homogenita rozptylů (Bartlett / Levene) - Jen pokud máme normalitu
bartlett <- bartlett.test(hodnota ~ skupina)
print("--- TEST ROZPTYLŮ (Bartlett) ---")
print(paste("Bartlett p-value:", format.pval(bartlett$p.value, digits=4)))
# p > 0.05 -> Rozptyly jsou SHODNÉ (var.equal = TRUE)
# p < 0.05 -> Rozptyly jsou RŮZNÉ (var.equal = FALSE) -> Welchův test

# 3. VÝBĚR A PROVEDENÍ TESTU
# ---------------------
# Nastav podle výsledků výše:
normalita_ok <- TRUE   # Změň na FALSE, pokud Shapiro vyšlo < 0.05
rozptyly_ok  <- TRUE   # Změň na FALSE, pokud Bartlett vyšel < 0.05
smer_testu   <- "two.sided" # "two.sided" (liší se), "greater" (1. > 2.), "less" (1. < 2.)

print("--- VÝSLEDEK TESTU ---")

if(normalita_ok) {
  if(rozptyly_ok) {
    print("✅ Parametrický T-TEST (Student - shodné rozptyly)")
    test_res <- t.test(hodnota ~ skupina, var.equal = TRUE, alternative = smer_testu)
  } else {
    print("⚠️ Parametrický T-TEST (Welch - různé rozptyly)")
    test_res <- t.test(hodnota ~ skupina, var.equal = FALSE, alternative = smer_testu)
  }
} else {
  print("🛑 Neparametrický MANN-WHITNEY U TEST (Wilcoxon Rank Sum)")
  test_res <- wilcox.test(hodnota ~ skupina, alternative = smer_testu)
}

print(test_res)

# 4. REPORT
if(test_res$p.value < 0.05) {
  print("✅ ZAMÍTÁME H0: Existuje statisticky významný rozdíl mezi skupinami.")
} else {
  print("❌ NEZAMÍTÁME H0: Rozdíl mezi skupinami není významný.")
}

In [ ]:
# ==============================================================================
# 🅱️ PÁROVÉ VÝBĚRY (Paired Test)
# ==============================================================================
# Kdy použít: "Liší se váha PŘED a PO dietě?", "Je rozdíl v hodnocení 1. a 2. kola?"
# Data: Dva číselné sloupce vedle sebe (x = Před, y = Po)
# ==============================================================================

# 1. NASTAVENÍ DAT
x_pred <- data$Vaha_Pred
y_po   <- data$Vaha_Po
rozdil <- x_pred - y_po # Spočítáme rozdíly

# 2. OVĚŘENÍ NORMALITY ROZDÍLŮ
# U párového testu nás zajímá jen normalita toho ROZDÍLU!
shapiro_diff <- shapiro.test(rozdil)
print(paste("Shapiro p-value (rozdílu):", format.pval(shapiro_diff$p.value, digits=4)))

# 3. PROVEDENÍ TESTU
smer_testu <- "two.sided" # "greater" pokud Pred > Po (zhubnul), "less" naopak

if(shapiro_diff$p.value > 0.05) {
  print("✅ Data normální -> PÁROVÝ T-TEST")
  test_pair <- t.test(x_pred, y_po, paired = TRUE, alternative = smer_testu)
} else {
  print("🛑 Data nenormální -> WILCOXON SIGNED RANK TEST")
  test_pair <- wilcox.test(x_pred, y_po, paired = TRUE, alternative = smer_testu)
}

print(test_pair)

In [ ]:
# ==============================================================================
# 📊 TEST SHODY DVOU PODÍLŮ (2-sample Proportions)
# ==============================================================================
# Kdy použít: "Mají muži větší nehodovost než ženy?", "Je Strategie A úspěšnější než B?"
# ==============================================================================

# 1. PŘÍPRAVA DAT (Musíme znát počty pro obě skupiny)
# Příklad: Strategie A vs B
data_A <- data %>% filter(Strategie == "A")
data_B <- data %>% filter(Strategie == "B")

# Počet úspěchů (x) a celkový počet (n) pro každou skupinu
x1 <- sum(data_A$Rocni_Vynosnost_v_Procentech > 0, na.rm=TRUE); n1 <- nrow(data_A)
x2 <- sum(data_B$Rocni_Vynosnost_v_Procentech > 0, na.rm=TRUE); n2 <- nrow(data_B)

print(paste("Skupina 1: Úspěchů", x1, "z", n1))
print(paste("Skupina 2: Úspěchů", x2, "z", n2))

# 2. PROVEDENÍ TESTU (prop.test)
# Vkládáme vektory c(x1, x2) a c(n1, n2)
res_prop <- prop.test(x = c(x1, x2), n = c(n1, n2), alternative = "two.sided")

print(res_prop)

# 3. INTERVALOVÝ ODHAD ROZDÍLU
print("Interval spolehlivosti pro rozdíl podílů:")
print(res_prop$conf.int)
# Pokud interval obsahuje 0, rozdíl není významný.

Jasně! ANOVA (Analýza rozptylu) tam bývá často, protože je to nejjednodušší způsob, jak porovnat **více než 2 skupiny** (např. 3 různé léky, 4 různé stroje).

Tady máš **"Copy-Paste" tahák pro Rko**. Zkopíruj si to do svého skriptu.

---

### 🟢 R-TAHÁK: ANOVA (Porovnání 3+ skupin)

**Kdy použít:** Mám jednu číselnou veličinu (např. *pevnost*) a jednu kategorickou (např. *typ_betonu* = A, B, C). Chci vědět, jestli se průměry liší.

#### 1. Příprava dat (Nutnost!)

Rko musí vědět, že ta skupina je kategorie, ne číslo.

```r
# Načtení dat (uprav si název souboru)
data <- read.csv("nazev_souboru.csv", sep=";", dec=",") 

# PŘEJMENOVÁNÍ PRO UNIVERZÁLNOST (aby fungoval zbytek kódu)
# data$y -> to co měříš (čísla)
# data$g -> to co rozděluje skupiny (A, B, C...)
data$y <- data$sloupec_s_cisly  # Např. data$pevnost
data$g <- as.factor(data$sloupec_se_skupinami) # Např. data$typ_stroje

# Rychlá vizualizace (pro kontrolu, zda to dává smysl)
boxplot(y ~ g, data = data, main = "Porovnání skupin")

```

#### 2. Ověření předpokladů (Bartlett & Shapiro)

Abych mohl použít ANOVU, musí platit:

1. **Normalita** (zbytky musí být normální).
2. **Homogenita rozptylů** (skupiny mají podobný rozptyl).

```r
# Vytvoření modelu (zatím jen pro testování předpokladů)
model <- aov(y ~ g, data = data)

# A) Test normality residuí (Shapiro-Wilk)
shapiro.test(residuals(model))
# Pokud p-value > 0.05 -> OK, data jsou normální.
# Pokud p-value < 0.05 -> ZAMÍTÁM normalitu -> měl bych použít Kruskal-Wallis (viz konec).

# B) Test shody rozptylů (Bartlettův test)
bartlett.test(y ~ g, data = data)
# Pokud p-value > 0.05 -> OK, rozptyly jsou shodné.
# Pokud p-value < 0.05 -> Rozptyly se liší -> ANOVA může být nepřesná (ale často se to ignoruje nebo se použije Welch).

```

#### 3. Samotná ANOVA (Pokud předpoklady platí)

Hypotézy:

* : Všechny průměry jsou stejné ().
* : Alespoň jeden průměr se liší.

```r
summary(model)

```

**Čtení výstupu:**

* Hledej sloupeček **`Pr(>F)`** (to je p-value).
* **p-value < 0.05**  **ZAMÍTÁM **. (Hurá, skupiny se liší, má to vliv).
* **p-value > 0.05**  Nezamítám . (Nic se neprokázalo, průměry jsou asi stejné).

#### 4. Post-hoc analýza (TukeyHSD)

Dělá se **POUZE**, pokud jsi v kroku 3 zamítl  (vyšlo ti, že se liší). Teď zjišťujeme, **KDO** se liší od koho.

```r
TukeyHSD(model)

```

**Čtení výstupu:**

* Dostaneš tabulku dvojic (např. A-B, A-C, B-C).
* Kde je **`p adj` < 0.05**, tam je statisticky významný rozdíl mezi těmito dvěma konkrétními skupinami.

---

### 🔴 ZÁCHRANA: Když neplatí normalita (Kruskal-Wallis)

Pokud ti v bodě 2 vyšel Shapiro-Wilk špatně (), použij neparametrickou verzi (mediány místo průměrů).

```r
kruskal.test(y ~ g, data = data)

```

* **p-value < 0.05**  Skupiny se liší.
* (Post-hoc pro Kruskala je složitější, obvykle u zkoušky stačí jen tento test).

---

### Stručný tahák na papír (Interpretace):

* **ANOVA :** Průměry všech skupin jsou shodné.
* **ANOVA :** Alespoň jeden průměr je jiný.
* **Předpoklady:** Normalita (Shapiro), Shoda rozptylů (Bartlett).
* **Výsledek:** Koukám na `Pr(>F)`. Je-li malá (< 0.05), dělám TukeyHSD.